In [3]:
import numpy as np
import pandas as pd
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [13]:
# 加载数据

def read_data():
    data = pd.read_csv('../data/VIP_day.csv', index_col='date',
                       usecols=['date', 'close', 'anger', 'disgust', 'joy', 'sadness', 'fear'])
#     print(data)
    return data.values

def create_dataset(X, y, look_back=1):
    dataX, dataY = [], []
    for i in range(len(y) - look_back - 1):
        dataX.append(dataset[i: (i + look_back)])
        dataY.append(y[i + look_back])
    return np.array(dataX), np.array(dataY)

In [21]:
# normalize the dataset
dataset = read_data()
y = dataset[:, 0]
X = dataset[:, 1:6]

for i in range(X.shape[0]):
    row_sum = sum(X[i, :])
    for j in range(X.shape[1]):
        X[i, j] /= row_sum

X = MinMaxScaler(feature_range=(-1, 1)).fit_transform(X)
print(X, y)


# split into train and test sets
train_size = int(len(X) * 2 / 3)
test_size = len(X) - train_size
X_train, X=test = X[:train_size, :], X[train_size:, :]
y_train, y=test = y[:train_size, :], y[train_size:, :]

# use this function to prepare the train and test datasets for modeling
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

print(trainX)
print(trainX.shape)
print(trainX[0][0])
# reshape input to be [samples, time steps, features]
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# print(trainX)

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(1, input_shape=(5, 12)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=1000, batch_size=10, verbose=2)

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


# invert predictions

trainPredict = scaler2.inverse_transform(trainPredict)
trainY = scaler2.inverse_transform(trainY)
testPredict = scaler2.inverse_transform(testPredict)
testY = scaler2.inverse_transform(testY)

print(trainPredict)
print(trainY)

print(len(list(trainY)), len(trainPredict))
trainScore = math.sqrt(mean_squared_error(list(np.reshape(trainY, (len(trainPredict), 1))), list(trainPredict)))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(list(np.reshape(testY, (len(testPredict), 1))), list(testPredict)))
print('Test Score: %.2f RMSE' % (testScore))


[[0.14991912 0.19302503 0.31748977 0.04971929 0.2898468 ]
 [0.14343063 0.17378255 0.35090418 0.05380056 0.27808209]
 [0.13515318 0.19439622 0.32476829 0.07574989 0.26993242]
 ...
 [0.0711993  0.10500646 0.35589228 0.02444871 0.44345325]
 [0.06492472 0.09814297 0.35804613 0.02713028 0.45175591]
 [0.09782494 0.11818325 0.37467164 0.02600609 0.38331407]] [ 3.11  0.58  4.32  1.32  2.81 -5.43  2.93 -0.48  0.42  0.52  2.31  1.31
 -0.11  1.67  0.61 -3.03 -1.98  3.36  2.77  0.33 -0.07  2.17  3.58  0.03
  0.67 -2.39 -0.24 -1.71  0.18 -0.4   3.54  1.2  -7.71  1.82  4.75  0.59
  0.25  0.94 -0.89 -1.41 -1.31 -1.59 -2.56  2.45 -0.96 -1.18 -1.93  0.62
  1.5   0.51  0.5   0.96  0.58  0.76 -0.56  2.15  0.36  0.79 -2.19  0.51
 -0.95 -0.22  1.89 -0.49  0.15  1.77  0.7   2.27  1.55  2.13  0.14  0.98
  1.95  0.1  -0.83  0.58  0.24  2.59 -1.02  1.67  0.41  1.    2.52  0.84
 -0.93  1.94  2.17  0.34 -1.24  2.71  2.2  -1.64  1.81  2.44  0.36 -0.47
  3.04 -1.13  0.01 -0.78  0.87 -4.06 -1.61 -2.77  2.28  3.04  

ValueError: Error when checking input: expected lstm_7_input to have shape (5, 12) but got array with shape (5, 6)